In [ ]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import matplotlib.pyplot as plt

from boostlets_mod import Boostlet_syst, indexs_max_conv_mask_dict, indexs_max_conv_mask_maskedDict, sk_to_phys
from mod_plotting_utilities import plot_array_images
from mod_RIRIS_func import load_DB_ZEA, rand_downsamp_RIR, ImageOps, jitter_downsamp_RIR
from mod_RIRIS_func import computePareto, ista, iffst, linear_interpolation_fft, perforMetrics


# Inputs 
Dictionary and Image

In [37]:
# Tamaño diccionario // Tamaño imagen interpolada
M, N  = 128, 128
# Imagen tamaño:
M0, N0 = 100, 100

# Dictionary
n_v_scales, n_h_scales = 10, 10  
base_v, base_h = 1/2, 1/2
n_v_thetas, n_h_thetas = 15, 15 

# Image
room = "Balder"
ratio_mics = 0.5
u = round(1/ratio_mics)
extrap_mode = "extrapolate" # or pad

#  subimage 
Tstart = 0
Tend = Tstart+M0

# sampling 
dx=3e-2
fs=11250
cs=340

# Pareto
beta_set = np.logspace(-2.5, -1, 50)

# ISTA
epsilon = 9.4e-6 # ISTA



## Create dict

In [ ]:
BS = Boostlet_syst(dx=dx, dt=1/fs, cs=cs,
                 M=M, N=N, 
                 n_v_scales=n_v_scales, n_h_scales=n_h_scales,
                 n_v_thetas=n_v_thetas, n_h_thetas=n_h_thetas, 
                 base_v=base_v, base_h=base_h, 
                 )

Sk = BS.get_boostlet_dict()

BS.print_max_scales()
# plot_array_images(Sk, num_cols=10)

In [ ]:
from mod_RIRIS_func import load_sk
folder_dict = 'saved_dicts/tan_dicts'
file_dict = 'BS_m_128_n_128_vsc_2_hsc_2_bases_0.5_0.5_thV_3_thH_3.mat'
Sk = load_sk(folder=folder_dict, file=file_dict, build_dict=None)
plot_array_images(Sk, num_cols=10)


## Load Image

In [ ]:
# ---------- LOAD Image ---------------------------
folder = "./dependencies/measurementData"
file = room+"RIR.mat"
file_path = os.path.join(folder, file)
print("Image loaded:")
print(file_path)

# Load full image and select a subimage to apply decomposition
full_image = load_DB_ZEA(file_path)[0]
orig_image = full_image[Tstart:Tend, :N0]

# mask0, _ = rand_downsamp_RIR(orig_image.shape, ratio_t=1, ratio_x=ratio_mics)
mask0, _ = jitter_downsamp_RIR(orig_image.shape, ratio_t=1, ratio_x=ratio_mics)


## Extrapolation

In [ ]:

# ----------------------------------------------------
# Extrapolation
# ----------------------------------------------------

extr_size = Sk.shape[:2]
imOps = ImageOps(orig_image.shape, mask=mask0, extrap_shape=extr_size, mode=extrap_mode) 
image = imOps.expand_image(orig_image)
mask = imOps.get_mask(image)

fft1 = np.fft.fftshift( np.fft.fft2(orig_image) )
fft2 = np.fft.fftshift( np.fft.fft2(image) )
fft3 = np.fft.fftshift( np.fft.fft2(mask*image) )

images = [orig_image, image, mask*image, abs(fft1), abs(fft2)+200*abs(Sk[:,:,12]), abs(fft3)]
titles = ['Original Image', 'Expanded Image', 'Masked image', r'$\mathcal{F}(orig. im)$', r'$\mathcal{F}(exp. im)$', r'$\mathcal{F}(mask. im)$']

fig, axs = plt.subplots(2,3, figsize=(6, 6))
for ax, im, titl in zip(axs.flatten(), images, titles):
    ax.pcolor(im[:128,:])
    ax.set_title(titl)
    ax.axis('off')
plt.tight_layout()
plt.show()

## Remove elements from dict

In [ ]:
from scipy import stats

id1, B1 = indexs_max_conv_mask_dict(1-mask, Sk, f_plot=False)
id2, B2 = indexs_max_conv_mask_maskedDict(1-mask, Sk, f_plot=False)


a = np.where(id1 != 0)[0] # indices de id1 donde id1 no es 0
boosts_id = id1[a] # id1 pero sin el indice boostlet0

z = stats.zscore(B1[boosts_id])

print(boosts_id)
print(z)

# zscore = (data_point - mean)/std_deviation
z = stats.zscore(B2[boosts_id]) # of the points excluding boostlet 0
temp = np.where(z > 0)[0] # point 1 std deviation above the mean
rm_idxs = boosts_id[temp]

print(rm_idxs)

In [8]:
# phys_sk = sk_to_phys(Sk)
# plot_array_images(phys_sk, cmap='gray', num_cols=10)

In [ ]:
# Sk1 = np.delete(Sk, [2, 11, 14, 17], axis=2) # NMSE = 4.3
# Sk1 = np.delete(Sk, [2], axis=2) # NMSE = -1.8
# Sk1 = np.delete(Sk, [14], axis=2) # NMSE = -3
# Sk1 = np.delete(Sk, [5], axis=2) # NMSE = -1.3
# Sk1 = np.delete(Sk, [8], axis=2) # NMSE = -5.7
# Sk1 = np.delete(Sk, [2,5,8], axis=2) # NMSE = -1.5
# Sk1 = np.delete(Sk, [11, 14, 17], axis=2) # NMSE = 2.7 (HORRIBLE)
# Sk1 = np.delete(Sk, [11, 14, 17, 1, 2, 3], axis=2) # NMSE = 5.4 (Pero tiene buena pinta)
# Sk1 = np.delete(Sk, [11, 14, 17, 1, 2, 3, 10, 12], axis=2) # NMSE = 4.34 (Mas o menos pero con ruido en la parte alta de la onda)
# Sk1 = np.delete(Sk, [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 14, 15, 17, 18], axis=2) # NMSE = 5.3 (Mas o menos pero con ruido en la parte alta de la onda) (No quitar el 12)



# Sk1 = np.delete(Sk, [2], axis=2)
# Sk1 = np.delete(Sk, rm_idxs, axis=2)
# Sk1 = np.delete(Sk, rm_idxs, axis=2)
Sk1 = np.delete(Sk, [2, 4], axis=2) # NMSE = -1.8


beta_set = np.logspace(-2.5, -0.5, 50)
beta_star, Jcurve = computePareto(image, mask, Sk1, beta_set, f_plot=True)
epsilon = 9.4e-6
alpha = ista(image, mask, Sk1, beta=beta_star, epsilon=epsilon, max_iterations=15, f_plot=False, f_verbose=False )
image_recov = iffst(alpha, Sk1)
final_image = imOps.recover_image(image_recov)
image_linear = linear_interpolation_fft(image*mask, dx=dx, fs=fs, cs=cs)
image_lin = imOps.recover_image(image_linear)
# %% Performance Metrics
NMSE_nlin, MAC, frqMAC = perforMetrics(image=image, image_recov=image_recov, 
                                    image_under=image*mask, 
                                    fs=fs, u=u, dx=dx, room=room)
images = [orig_image[:100,:], (orig_image*mask0)[:100,:], final_image[:100,:], image_lin[:100,:]]
titles = ['Original Image', 'Masked Image', 'Final reconst image', "Linear reconst"]
fig, ax = plt.subplots(1, len(images), figsize=(18, 6))
for i in range(len(images)):
    ax[i].imshow(images[i])
    ax[i].set_title(titles[i])
    ax[i].axis('off')
plt.tight_layout()
plt.suptitle(f"NMSE lin = {NMSE_nlin[0]};    NMSE dict = {NMSE_nlin[1]}")
plt.show()



In [ ]:
print(beta_star)
print(Jcurve)

# ISTA


In [ ]:
# ----------------------------------------------------
# Pareto
# ----------------------------------------------------
beta_star, Jcurve = computePareto(image, mask, Sk, beta_set, f_plot=True)

# %%

# ----------------------------------------------------
# ISTA recovery
# ----------------------------------------------------
alpha = ista(image, mask, Sk, beta=beta_star, epsilon=epsilon, max_iterations=15, f_plot=True, f_verbose=True )


## Recover image

In [10]:
# recover inpainted image from sparse coefficients (Eq. 19)
image_recov = iffst(alpha, Sk)
final_image = imOps.recover_image(image_recov)


## Linear interpolation

In [11]:
image_linear = linear_interpolation_fft(image*mask, dx=dx, fs=fs, cs=cs)
image_lin = imOps.recover_image(image_linear)


# %% Performance Metrics
NMSE_nlin, MAC, frqMAC = perforMetrics(image=image, image_recov=image_recov, 
                                    image_under=image*mask, 
                                    fs=fs, u=u, dx=dx, room=room)


## Visual results

In [ ]:

images = [orig_image[:100,:], (orig_image*mask0)[:100,:], final_image[:100,:], image_lin[:100,:]]
titles = ['Original Image', 'Masked Image', 'Final reconst image', "Linear reconst"]

fig, ax = plt.subplots(1, len(images), figsize=(18, 6))
for i in range(len(images)):
    ax[i].imshow(images[i])
    ax[i].set_title(titles[i])
    ax[i].axis('off')
plt.suptitle(f"NMSE lin = {NMSE_nlin[0]};    NMSE dict = {NMSE_nlin[1]}")
plt.tight_layout()
plt.show()